# AlphaQUBO Tutoral - S3 Compressed Sampler

This tutorial shows how to read an AlphaQUBO formatted file, process it as a D-Wave BinaryQuadraticModel, and submit to an AlphaQUBO solver.

The AlphaQUBO file format is a text file with information on the first line that describes the size of the q(i,j) matrix.

1. Program line is marked by a “**p**” in the first column. A single program line must be the first line in the file. The program line has two arguments: **variable_count** and **non_zero_element_count**

```
    p 50 225
```

2. The remaining lines are made up of three numbers, separated by one or more blanks. The first two numbers, ($i$ and $j$), are the indices for this Q matrix entry, where ($i <= j$). Each index must be in the range {1, **variable_count**}. The third number is the value of the Q matrix at $Q(i,j)$, specified as an integer or floating point value

#### Example
```
 p  100  475
 1  35  -19
 1  44  -22
 1  47  27
 1  49  -66
 1  58  -69
 1  64  63
 1  72  -89
 1  73  -19
 1  74  -69
 1  76  -12
 1  84  40
 1  98  33
 2  2  52
```


In [1]:
import sys
import alphaqubo_client as aq
from alphaqubo_client.rest import ApiException
import dimod
import boto3
from botocore.exceptions import ClientError
import gzip

## Read AlphaQUBO formatted file as BinaryQuadraticModel

In [2]:
def read_alphaqubo_bqm(filename:str):
    lines = []
    with open(filename, "rt") as myfile:
        for line in myfile:
            lines.append(line.rstrip('\n'))
    Q = {}
    for line in lines:
        p = line.split()
        if len(p) > 0:
            if p[0] == 'c':
                continue
            elif p[0] == 'p':
                nVars = int(p[1])
            else:
                if len(p) == 3:
                    i = int(p[0]) - 1
                    j = int(p[1]) - 1
                    w = float(p[2])
                    if i != j:
                        w *= 2
                    Q[(i, j)] = w

    bqm = dimod.BinaryQuadraticModel.from_qubo(Q)
    return bqm

## Configure AlphaQUBO API
Configure the connection details to connect to AlphaQUBO SolverAPI

In [3]:
configuration = aq.Configuration()
configuration.debug = False
configuration.host = "http://ec2-54-201-35-39.us-west-2.compute.amazonaws.com:9000"

api_instance = aq.QuboApi(aq.ApiClient(configuration))

### Read AlphaQUBO Formatted File


In [4]:
# BUCKET_NAME = 'qubotests'
# FILENAME = 'bqp2500_1.gz'

bqm = read_alphaqubo_bqm('../data/bqp2500_1.txt')
print("Size of BQM: ", len(bqm))


Size of BQM:  2500


### AlphaQUBO S3 dimod Sampler

In [5]:
from dimod.core.sampler import Sampler
from dimod.sampleset import SampleSet
import json

class AlphaQuboS3Sampler():

    properties = None
    parameters = None
    api_instance = None
    bucket_name = 'alphaqubo'
    name = None
    region = 'us-west-2'

    def __init__(self, api_instance, region, bucket_name, name):
        self.properties = {}
        self.parameters = {'time_limit': [],
                           'accuracy_min': [],
                           'greediness': [],
                           'maximize': [] }
        self.api_instance = api_instance
        self.bucket_name = bucket_name
        self.name = name

    def sample(self, bqm, time_limit=60, accuracy_min=5, greediness=0.0, maximize=False):
        n = len(bqm.variables)  
        filename = self.name + '.gz'
        body = aq.SolverAsyncRequest()
        body.num_vars = n
        body.region = self.region
        body.bucket_name = self.bucket_name
        body.key_name = filename
        body.solution_bucket_name = self.bucket_name
        body.solution_key_name = self.name + '.sol'
        
        if maximize:
            body.min_max = 1
        else:
            body.min_max = 0
        body.timeout = time_limit
        
        body.parameters = "-am " + str(accuracy_min)
        if greediness > 0.0:
            body.parameters = body.parameters + '-g ' + str(greediness)

        self.save_alphaqubo_gzip(filename, bqm)
        self.upload_file( filename, self.bucket_name, object_name=filename )
            
        try:
            # solve it asynchronously.
            api_response = api_instance.api_qubo_solve_qubo_async_using_s3_post(body=body)           
        except ApiException as e:
            print("Exception when calling QuboApi->api_qubo_solve_qubo_async_using_s3_post: %s\n" % e)
        
        return api_response
        
    def job_results(self):
        solfile = self.name + '.sol'
        self.download_file(self.bucket_name, solfile)
        with open(solfile) as f:
            data = json.load(f)
        
        samples = []
        energies = []
        samples.append( data['solvedResult'] )
        energies.append( data['solvedValue'] )
              
        response = SampleSet.from_samples(samples, bqm.vartype, energy=energies)
        return response
        
    
    def save_alphaqubo_gzip(self, filename:str, bqm):
        with gzip.open( filename, "wt" ) as fl:
            variables = sorted(bqm.iter_variables())  # integer labeled so we can sort in py3
            # l = dict(filter(lambda x: x != 0, bqm.linear.items()))  # filter out zero linear terms
            nonzero = 0
            for idx, u in enumerate(variables):
                for v in variables[idx:]:
                    if u == v and bqm.linear[u]:
                        nonzero += 1
                    if u in bqm.adj[v]:
                        nonzero += 1
            
            print("p", bqm.num_variables,  nonzero, file=fl)
            for idx, u in enumerate(variables):
                for v in variables[idx:]:
                    if u == v and bqm.linear[u]:
                        print(u+1, u+1, bqm.linear[u], file=fl)
                    if u in bqm.adj[v]:
                        print( u+1, v+1, bqm.adj[u][v]/2, file=fl)

    def upload_file(self, file_name, bucket, object_name=None):
        """Upload a file to an S3 bucket

        :param file_name: File to upload
        :param bucket: Bucket to upload to
        :param object_name: S3 object name. If not specified then file_name is used
        :return: True if file was uploaded, else False
        """

        # If S3 object_name was not specified, use file_name
        if object_name is None:
            object_name = file_name

        # Upload the file
        s3_client = boto3.client('s3')
        try:
            response = s3_client.upload_file(file_name, bucket, object_name, ExtraArgs={'ContentType': 'application/x-gzip'})
        except ClientError as e:
            print(e)
            return False
        return True
    
    def download_file(self, bucket, file_name):
        s3 = boto3.client('s3')
        s3.download_file(bucket, file_name, file_name)

### Solve using AlphaQUBO

In [6]:
sampler = AlphaQuboS3Sampler(api_instance, 'us-west-2', 'alphaqubo-test', 'qtest')
response = sampler.sample(bqm, maximize=True, time_limit=15)
print(response)
import time
time.sleep(15+10)

{'bucket_name': 'alphaqubo-test',
 'qubo_accepted': True,
 'qubo_key_name': 'qtest.gz',
 'region': 'us-west-2',
 'solution_bucket_name': 'alphaqubo-test',
 'solution_httpurl': 'https://s3.us-west-2.amazonaws.com/alphaqubo-test/qtest.sol',
 'solution_key_name': 'qtest.sol',
 'solution_s3_uri': 's3://alphaqubo-test/qtest.sol'}


In [7]:
sampler = AlphaQuboS3Sampler(api_instance, 'us-west-2', 'alphaqubo-test', 'qtest')
results = sampler.job_results()
print(results)

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 ... 2499  energy num_oc.
0  1  1  1  1  1  1  1  1  1  1  0  0  0  0  0  0  0 ...    1 1515723       1
['BINARY', 1 rows, 1 samples, 2500 variables]
